# WESTPA-WExplore plugin workshop
---
*WESTPA Workshop, University of Pittsburgh, Aug 2 2018*  
by  
Alex Dickson, Michigan State University  
Audrey Pratt, University of Pittsburgh

**Special thanks to Joshua Adelman for doing the heavy lifting back in 2014!**

---

## Overview

This workshop will describe the installation, setup, runtime and analysis of the application of the `WESTPA-WExplore` plugin.  As a model system, we will use the $K^+$/18-Crown-6 Ether system from [this publication](https://pubs.acs.org/doi/abs/10.1021/ct100626x). 

![system](system-01.png)

### Dependencies

**If you are running this on H2P, then the install scripts will take care of everything.**  If you are following along on your laptop, or another machine you will need the following:

- `GROMACS` to run dynamics on CPUs.  [It can be found here](http://www.gromacs.org/Downloads).  
- `WESTPA` should already be installed.  [Link](https://github.com/westpa/westpa/wiki/Installing-WESTPA)
- A `conda` environment will be created to run this example. [Download anaconda python here](https://conda.io/docs/user-guide/install/download.html) 

We assume that the user has some familiarity with bash and python.

### A note on this notebook:

This will give commands that must be executed in bash like so:
```bash
~/WESTPA-WExplore/examples/kcrownether> echo "this is a bash command"
```
where `~/WESTPA-WExplore/examples/kcrownether` is the directory from which this command should be run.  Later on in this notebook there will be some cells with python code which can be run directly from this `jupyter` notebook, but these require that the environment is properly set, so it might be required to close and re-open this notebook after the `WESTPA-WExplore` plugin is properly installed.

---

## Installation

To install the plugin, clone the repository from your home directory, or from where you wish to install the plugin:
```bash
~> git clone https://github.com/ADicksonLab/WESTPA-WExplore.git
```
change to the WESTPA-WExplore directory:
```bash
~> cd WESTPA-WExplore
```
create the `conda` environment and install the package as follows:
```bash
~/WESTPA-WExplore> source create_conda_environment.sh
```
This last command will require you to agree to a lot of things.  **Note that if you are on your own machine, you can likely comment out the `module load python/anaconda2.7-4.2.0` command**.

---
## K-crown-ether example

Now switch to the kcrownether directory:
```bash
~/WESTPA-WExplore> cd examples/kcrownether
```
and let's look at some of the files here, and see how we can change parameters that are unique to the `WESTPA-WExplore` plugin.

### Distance metric

The distance metric here is the "ligand unbinding" distance metric, where the ligand is the K$^+$ atom.  The distance metric is written as the function `dfunc` in the file `system.py`:

```python
def dfunc(p, centers):
    d = np.array([np.linalg.norm((p - c)) for c in centers], dtype=np.float32)

    # the above is the Frobenius norm, for RMSD divide by N^1/2, where N is number of atoms      
    # (Note that p has length 3*N)
    d /= math.sqrt(len(p)/3)

    return d
```

The `p` array here is the coordinates of the K+ atom (e.g. the X, Y, Z positions), after alignment to the crownether.  These are the `pcoord` values that will be saved to `west.h5`.

### Region sizes

Here we use a three-level hierarchy of Voronoi polyhedra, which are each defined by an "image".  The coordinates of these regions will also be saved to `west.h5`.  The region hierarchy is also set up in `west.cfg`:

```python
class System(WESTSystem):
    def initialize(self):
        # The number of dimensions should be the number of atoms that we have multipled by 3.                                    
        self.pcoord_ndim = 3
        self.pcoord_len = 11
        self.pcoord_dtype = np.float32

        self.bin_mapper = wexplore.WExploreBinMapper(n_regions=[10,10,10], d_cut=[5, 2.0, 0.8], dfunc=dfunc)
        # The initial center is on the coordinates of one of the basis states.                                                   
        init_struct = np.loadtxt('18-crown-6-K+.pdb', dtype=str)
        atom_coords = init_struct[5:8]
        atom_coords = atom_coords.astype(float).flatten()
        self.bin_mapper.centers = [atom_coords]
        self.bin_mapper.add_bin(None, 0)
        self.max_replicas = 48
        self.bin_target_counts = self.bin_mapper.balance_replicas(self.max_replicas,
                                np.array([0,], np.int_))
```
The bin mapper initialization contains both the distance cutoffs (`d_cut`) and the maximum branching numbers (`n_regions`) at each level of the hierarchy.  To initialize the bin mapper we need a single region, we use the coordinates of the K$^+$ ion from `18-crown-6-k+.pdb`.

### Extracting coordinates for the distance metric

A big challenge is extracting information from the MD engine (here, `GROMACS`) to use to compute our distances.  Here we need to align structures to a template using the crown-ether atoms, and then get the coordinates of the ligand, which in our case is the single K$^+$ atom.  These coordinates are then fed to the `pcoord_loader` in `system.py`.  

The coordinate extraction happens in three different places: `init.sh` (to get the initial image), `get_pcoord.sh` and `runseg.sh`.  Here is the code from `runseg.sh`:

```bash
echo -e "4 \n" | gmx trjconv    -f nojump.xtc  -s seg.tpr -n $NDX -o pcoord.pdb || exit 1

# Copy in the imaged trajectory as the progress coordinate.  We'll use a python  
# pcoord loader to analyze the RMSD and go from there.                                                                           
echo "2 9" | gmx trjconv -fit rot+trans -s bound_state.tpr -f pcoord.pdb -o pcoord_align.pdb
cat pcoord_align.pdb | grep '^ATOM' | grep K\+ > $WEST_PCOORD_RETURN || exit 1
```

### Running the example

OK let's go!  First, we initialize the run environment:
```bash
~/WESTPA-WExplore/examples/kcrownether> ./init.sh
```
then we submit a job using SLURM:
```bash
~/WESTPA-WExplore/examples/kcrownether> sbatch ./runwe_h2p.sh
```

Or, if you are running locally, you can directly execute the run script:
```bash
~/WESTPA-WExplore/examples/kcrownether> ./run.sh --parallel --n_workers 8
```

---
## K-crown-ether analysis

Great!  Now that our job has finished, we can look at `west.log`.  If you ran a long run, you might have output that looks like this:

```
Fri Jul 27 00:02:23 2018
Iteration 300 (300 requested)
Beginning iteration 300
48 segments remain in iteration 300 (48 total)
45 of 45 (100.000000%) active bins are populated
per-bin minimum non-zero probability:       4.92423e-14
per-bin maximum probability:                0.850158
per-bin probability dynamic range (kT):     30.4797
per-segment minimum non-zero probability:   4.92423e-14
per-segment maximum non-zero probability:   0.486652
per-segment probability dynamic range (kT): 29.9218
norm = 1, error in norm = 0 (0*epsilon)
--wexplore-stats--------------------
wallclock time: 0.655 s

Level 0: 10 cells (10 max)
Level 1: 100 cells (100 max)
Level 2: 978 cells (1000 max)
------------------------------------
Iteration wallclock: 0:01:19.324861, cputime: 0:13:44.526626


Fri Jul 27 00:03:42 2018
WEST run complete.
```

The `wexplore-stats` section shows how many regions were created.

WExplore used a three-level hierarchy of Voronoi polyhedra, which are each defined by an "image". Here we save the coordinates of all the images in the variables `big_regions`, `med_regions` and `sml_regions`.  We measure distances between the different sets of images, and we end the demonstration by writing trajectories containing these coordinates to "dcd" files, which can be visualized in VMD or pymol.

**Note: If your simulation didn't work and you want to analyze a pre-made one, unpack `long_run.tgz` and change to the `long_run` directory:**

```bash
tar xzf long_run.tgz
cd long_run
```

---
To run some analysis code, let's import some things, and add some `westpa` things to the path:

In [2]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

import os
import sys

WEST_ROOT = os.environ['WEST_ROOT']
for lib in ['lib/wwmgr', 'src', 'lib/west_tools']:
    path = os.path.join(WEST_ROOT, lib)
    if path not in sys.path:
        sys.path.append(path)

import west
        
%matplotlib inline

The data manager helps access simulation data written to the `west.h5` file:

In [3]:
data_manager = west.rc.get_data_manager()
data_manager.open_backing(mode='r')

In [5]:
string_hashes = data_manager.we_h5file['bin_topologies']['index'][:]['hash']
all_mappers = []
for si, shash in enumerate(string_hashes):
    bin_mapper = data_manager.get_bin_mapper(shash)
    all_mappers.append(bin_mapper)

Each mapper holds the coordinates of all of the images.  Let's examine the latest one:

In [6]:
latest_mapper = all_mappers[-1]
level_inds = latest_mapper.level_indices

In [14]:
# get region indices
biggest_regions_inds = level_inds[0]
medium_regions_inds = level_inds[1]
small_regions_inds = level_inds[2]

# find region centers
big_regions = latest_mapper.fetch_centers(biggest_regions_inds)
med_regions = latest_mapper.fetch_centers(medium_regions_inds)
sml_regions = latest_mapper.fetch_centers(small_regions_inds)

print("There are {0} big regions, {1} medium regions, and {2} large regions"
      .format(len(big_regions),len(med_regions),len(sml_regions)))

There are 10 big regions, 100 medium regions, and 978 large regions


Now let's calculate the distance between all of the medium regions:

In [12]:
from system import dfunc

n_med = len(med_regions)
dists = np.zeros((n_med,n_med))
for i, x in enumerate(med_regions):
    for j, y in enumerate(med_regions):
        dists[i,j] = dfunc(x,[y])

In [13]:
dists

array([[  0.        ,   2.0325439 ,   2.16072845, ...,  11.98263073,
         14.05814457,   9.71697235],
       [  2.0325439 ,   0.        ,   3.94243813, ...,  10.60643768,
         12.48051739,  10.04217911],
       [  2.16072845,   3.94243813,   0.        , ...,  12.94781685,
         15.27670479,   8.89304543],
       ..., 
       [ 11.98263073,  10.60643768,  12.94781685, ...,   0.        ,
          2.8847928 ,   9.76592064],
       [ 14.05814457,  12.48051739,  15.27670479, ...,   2.8847928 ,
          0.        ,  12.63375282],
       [  9.71697235,  10.04217911,   8.89304543, ...,   9.76592064,
         12.63375282,   0.        ]])

Note that none of the images are closer than the `d_cut` value for the medium regions (2.0 A).

In [10]:
# great!  now let's write these to trajectories
import mdtraj as mdj

ion_pdb = mdj.load_pdb('18-crown-6-K+.pdb')

big_reg_reshape = big_regions.reshape((len(big_regions),ion_pdb.n_atoms,3))/10
med_reg_reshape = med_regions.reshape((len(med_regions),ion_pdb.n_atoms,3))/10
sml_reg_reshape = sml_regions.reshape((len(sml_regions),ion_pdb.n_atoms,3))/10

mdj.Trajectory(big_reg_reshape,ion_pdb.top).save_dcd('big_regions.dcd')
mdj.Trajectory(med_reg_reshape,ion_pdb.top).save_dcd('med_regions.dcd')
mdj.Trajectory(sml_reg_reshape,ion_pdb.top).save_dcd('sml_regions.dcd')

### Great, now go ahead and visualize your regions!  They are pre-aligned.

There is a helpful VMD script to accomplish this:

```
vmd -e vis_align.tcl
```

### After running this script for a short run, and doing a little decorating, we get this:

![](all_regions-01.png)

---
## Bonus: Ring Potential example

There is a great example included in the initial WESTPA-WExplore repo of a Langevin particle diffusing around a ring.  This is also included in the `examples` folder.  To run, first build the cython extensions:
```
~/WESTPA-WExplore> cd examples/RingPotential
~/WESTPA-WExplore/examples/RingPotential> python setup.py build_ext --inplace
```
and then run `init.sh` and `run.sh`.

That folder contains a great analysis script to visualize Voronoi polyhedra (written by Josh).  I recommend copying over the simulation results and `analysis.ipynb` to your local machine (if you have `ipython` installed) and launching the notebook in your browser:
```
ipython notebook
```